In [2]:
# 📌 IMPORTACIONES (Todas al inicio)
from google.colab import drive
import os
import pandas as pd
import multiprocessing
import pickle
from collections import defaultdict
from tqdm.notebook import tqdm


# 📌 MONTAR GOOGLE DRIVE
drive.mount('/content/drive')

# 📌 Ruta base donde están los archivos
ruta_base = "/content/drive/MyDrive/Bases_Iniciales/"


# 📌 FUNCIÓN PARA LEER ARCHIVOS EXCEL
def leer_archivo_excel(args):
    file_path, sheet_name, skiprows, key = args
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)
        return key, df
    except Exception as e:
        print(f"❌ Error en {key}: {e}")
        return key, None


# 📌 Diccionario para almacenar DataFrames
dataframes = {}

# 📌 LISTA DE TAREAS PARA MULTIPROCESSING
tareas = []

# 📌 Definir periodos correctamente
periodos = {
    "📂 Periodo 1 - Años 1995-2003, 2007": list(range(1995, 2004)) + [2007],
    "📂 Periodo 2 - Años 2004-2015 (excepto 2007)": [year for year in range(2004, 2016) if year != 2007],
    "📂 Periodo 3 - Años 2016-2023": list(range(2016, 2024))  # Incluye 2023
}

# 📌 AÑOS 1995-2003 y 2007
for year in periodos["📂 Periodo 1 - Años 1995-2003, 2007"]:
    file_path_bg = os.path.join(ruta_base, f'{year}EstadosFinancieros.xlsx')
    file_path_fe = os.path.join(ruta_base, f'{year}EstadosFinancieros.xlsx')
    tareas.append((file_path_bg, f'BG-ER{year}', 2 if year == 2007 else 1, f'{year}_BG_ER'))
    tareas.append((file_path_fe, f'FE-{year}', 1, f'{year}_FE'))

# 📌 AÑOS 2004-2015 (excepto 2007)
for year in periodos["📂 Periodo 2 - Años 2004-2015 (excepto 2007)"]:
    file_path_bg = os.path.join(ruta_base, f'{year}BalanceGeneral.xlsx')
    file_path_er = os.path.join(ruta_base, f'{year}EstadoResultados.xlsx')
    file_path_fe = os.path.join(ruta_base, f'{year}FlujoEfectivo.xlsx')
    tareas.append((file_path_bg, f'BalanceGeneral{year}SIREMV2', None, f'{year}_BG'))
    tareas.append((file_path_er, f'EstadoResultados{year}SIREMV2', None, f'{year}_ER'))
    tareas.append((file_path_fe, f'FlujoEfectivo{year}SIREMV2', None, f'{year}_FE'))

# 📌 AÑOS 2016-2023 (Pymes y Plenas)
for year in periodos["📂 Periodo 3 - Años 2016-2023"]:
    for tipo, sigla in [("PymesCaratula", "PY_CA"), ("PymesEstadodeflujosdeefectivometodoindirecto", "PY_FE"),
                        ("PymesEstadodelresultadointegralcomponentesORIpresentadosnetosdeimpuestos", "PY_ORI"),
                        ("PymesEstadoderesultadointegralresultadodelperiodoporfunciondegasto", "PY_ER"),
                        ("PymesEstadodesituacionfinancieracorrientenocorriente", "PY_SF"),
                        ("PlenasCaratula", "PL_CA"), ("PlenasEstadodeflujosdeefectivometodoindirecto", "PL_FE"),
                        ("PlenasEstadodelresultadointegralcomponentesORIpresentadosnetosdeimpuestos", "PL_ORI"),
                        ("PlenasEstadoderesultadointegralresultadodelperiodoporfunciondegasto", "PL_ER"),
                        ("PlenasEstadodesituacionfinancieracorrientenocorriente", "PL_SF")]:
        tareas.append((os.path.join(ruta_base, f'{year}{tipo}.xlsx'), 'Sheet1', None, f'{year}_{sigla}'))

# ✅ PROCESAMIENTO CON PROGRESO
from multiprocessing import Pool

resultados = []
with Pool(processes=num_procesos) as pool:
    for r in tqdm(pool.imap_unordered(leer_archivo_excel, tareas), total=len(tareas), desc="📥 Cargando archivos"):
        resultados.append(r)


# 📌 GUARDAR RESULTADOS EN EL DICCIONARIO
for key, df in resultados:
    dataframes[key] = df

# 📌 RESUMEN DE ARCHIVOS CARGADOS
archivos_cargados = len([df for df in dataframes.values() if df is not None])
print(f"\n📊 Se han leído correctamente {archivos_cargados} archivos de un total de {len(tareas)}.")

# 📌 ARCHIVOS FALTANTES
expected_keys = [tarea[3] for tarea in tareas]
missing_files = [key for key in expected_keys if key not in dataframes]
if missing_files:
    print("\n❌ Archivos que NO se cargaron:")
    for missing in missing_files:
        print(f"⚠️ {missing}")
    print("\n🔴 SOLUCIÓN: Verifica si estos archivos están en Google Drive y sus hojas son correctas.")
else:
    print("\n✅ Todos los archivos se cargaron correctamente.")

# 📌 RESUMEN POR PERIODO
resumen_por_periodo = defaultdict(lambda: defaultdict(list))
for key in sorted(dataframes.keys()):
    año = int(key.split("_")[0])
    for periodo, años in periodos.items():
        if año in años:
            resumen_por_periodo[periodo][año].append(key)

# 📌 IMPRIMIR RESUMEN
print("\n📊 Resumen de archivos cargados por periodo:\n")
for periodo, años_dict in resumen_por_periodo.items():
    total_archivos = sum(len(archivos) for archivos in años_dict.values())
    print(f"{periodo} ({total_archivos} archivos cargados)\n")
    for año, archivos in sorted(años_dict.items()):
        archivos_ordenados = sorted(archivos)
        archivos_separados = " | ".join(archivos_ordenados)
        print(f"   ✅ Año {año} → {archivos_separados} ({len(archivos)} cargados)")
    print()

# 📌 GUARDAR DATAFRAMES EN .PKL
ruta_guardado = "/content/drive/MyDrive/Datos/1_ArchivosLeidos.pkl"
os.makedirs(os.path.dirname(ruta_guardado), exist_ok=True)
with open(ruta_guardado, "wb") as f:
    pickle.dump(dataframes, f)
print(f"\n✅ DataFrames guardados en: {ruta_guardado}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


📥 Cargando archivos:   0%|          | 0/133 [00:00<?, ?it/s]


📊 Se han leído correctamente 133 archivos de un total de 133.

✅ Todos los archivos se cargaron correctamente.

📊 Resumen de archivos cargados por periodo:

📂 Periodo 1 - Años 1995-2003, 2007 (20 archivos cargados)

   ✅ Año 1995 → 1995_BG_ER | 1995_FE (2 cargados)
   ✅ Año 1996 → 1996_BG_ER | 1996_FE (2 cargados)
   ✅ Año 1997 → 1997_BG_ER | 1997_FE (2 cargados)
   ✅ Año 1998 → 1998_BG_ER | 1998_FE (2 cargados)
   ✅ Año 1999 → 1999_BG_ER | 1999_FE (2 cargados)
   ✅ Año 2000 → 2000_BG_ER | 2000_FE (2 cargados)
   ✅ Año 2001 → 2001_BG_ER | 2001_FE (2 cargados)
   ✅ Año 2002 → 2002_BG_ER | 2002_FE (2 cargados)
   ✅ Año 2003 → 2003_BG_ER | 2003_FE (2 cargados)
   ✅ Año 2007 → 2007_BG_ER | 2007_FE (2 cargados)

📂 Periodo 2 - Años 2004-2015 (excepto 2007) (33 archivos cargados)

   ✅ Año 2004 → 2004_BG | 2004_ER | 2004_FE (3 cargados)
   ✅ Año 2005 → 2005_BG | 2005_ER | 2005_FE (3 cargados)
   ✅ Año 2006 → 2006_BG | 2006_ER | 2006_FE (3 cargados)
   ✅ Año 2008 → 2008_BG | 2008_ER | 2008_FE